* **download hf model**  
* method-1: snapshot_download 

<!-- 
- 纯粹下载模型仓库的所有文件，不会加载模型或初始化任何HuggingFace的高层对象
- 用途：适合离线部署、镜像备份、完整复制模型目录。
- 输出：返回一个本地路径（通常是缓存目录），你可以手动移动或指定 local_dir。
- 不加载模型，不涉及 transformers，因此不会触发模型结构检查或依赖项验证 
-->

只想下载模型文件（如部署到服务器）：用 snapshot_download

In [ ]:
import os 
from huggingface_hub import snapshot_download
# 1. 设置镜像源（国内加速）
# os.environ["HF_ENDPOINT"] = "https://mirrors.tuna.tsinghua.edu.cn/hugging-face/"
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# 2. 定义模型列表（名称 + 下载路径）
# os.path.expanduser("./xx") 会将 ~ 替换为当前用户的主目录
models_to_download = [
    {
        "repo_id": "Qwen/Qwen3-Embedding-0.6B",  # Embedding 模型 bge-m3 Qwen/Qwen3-Embedding-0.6B
        "local_dir": os.path.expanduser("./mineru_models/Qwen3-Embedding-0.6B"),
    },
    {
        "repo_id": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # LLM 模型
        "local_dir": os.path.expanduser("./mineru_models/DeepSeek-R1-1.5B"),
    }
]

# 3. 遍历下载所有模型
for model in models_to_download:
    while True:  # 断点续传重试机制
        try:
            print(f"开始下载模型: {model['repo_id']} 到目录: {model['local_dir']}")
            snapshot_download(
                repo_id=model["repo_id"],
                local_dir=model["local_dir"],
                resume_download=True,  # 启用断点续传
                force_download=False,  # 避免重复下载已有文件
                token=None,            # 如需访问私有模型，替换为你的 token
            )
            print(f"模型 {model['repo_id']} 下载完成！")
            break
        except Exception as e:
            print(f"下载失败: {e}, 重试中...")


* **download hf model**
* method-2: AutoTokenizer.from_pretrained(...)
<!-- 
作用：下载 + 加载模型/分词器，并返回可直接使用的 Python 对象。
用途：适合快速开发、实验、推理或微调。
行为：首次运行会自动下载模型文件到缓存目录；后续运行直接从缓存加载。
会检查模型结构、依赖、配置是否正确，并加载权重 
-->

想直接加载模型做推理或训练：用 AutoTokenizer + AutoModel.from_pretrained

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# default: c/user/.cache/huggingface/transformers
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)   
model = AutoModelForCausalLM.from_pretrained(model_name)

# saved = r"./my_model"  
# tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=saved, trust_remote_code=True)   
# model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=saved)

# model.save_pretrained("./my_model")
# tokenizer.save_pretrained("./my_model")